In [2]:
import pandas as pd
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil
from scipy.stats import norm
import statistics
import math

In [3]:
# dataset

In [4]:
df = pd.read_csv('AB_test_data.csv')

In [5]:
df.head()

,purchase_TF,Variant,date,id
0,False,A,2019-11-08,0x25b44a
1,False,B,2020-08-27,0x46271e
2,False,A,2020-06-11,0x80b8f1
3,False,B,2020-08-22,0x8d736d
4,False,A,2020-08-05,0x96c9c8


In [6]:
pd.crosstab(df['purchase_TF'], df['Variant'])

Variant,A,B
purchase_TF,,
False,106298,4117
True,18702,883


In [7]:
# Question 1

In [8]:
variant_A = df[df['Variant'] == 'A']
variant_B = df[df['Variant'] == 'B']

In [9]:
mean_a = sum(variant_A['purchase_TF'])/len(variant_A['purchase_TF'])
mean_b = sum(variant_B['purchase_TF'])/len(variant_B['purchase_TF'])
print(mean_a,mean_b)

0.149616 0.1766


In [10]:
z_score = (mean_b - mean_a)/pow((mean_a*(1-mean_a))/len(variant_B),0.5) #one-sample z test
print(f'z score: {z_score:.2f}')
if z_score >= 1.64:
    print('Reject null hypothesis.')
else:
    print('Fail to Reject null hypothesis.')

z score: 5.35
Reject null hypothesis.


In [11]:
# Question 2

In [14]:
# using a significance level of 0.05, and setting power to 0.80, we have tα/2 = 1.96 and tβ = 0.84 from standard normal tables.
tα_2 = 1.96
tβ = 0.84
pool_variance = mean_a*(1-mean_a)+mean_b*(1-mean_b)
n_1 = ceil((pool_variance/(mean_a-mean_b)**2)*((tβ+tα_2)**2))
n_1

2936

In [15]:
# sample and test

In [16]:
for i in range(10,20):
    sample_B = variant_B.sample(n=n_1)
    conversion_b = sum(sample_B['purchase_TF'])/len(sample_B['purchase_TF'])
    z_score = (conversion_b - mean_a)/pow((mean_a*(1-mean_a))/len(sample_B),0.5)
    if z_score >= 1.64:
        print(f'z_score: {z_score:.2f}','Reject null hypothesis.')
    else:
        print(f'z_score: {z_score:.2f}','Fail to Reject null hypothesis.')

z_score: 3.71 Reject null hypothesis.
z_score: 5.94 Reject null hypothesis.
z_score: 4.38 Reject null hypothesis.
z_score: 4.23 Reject null hypothesis.
z_score: 4.49 Reject null hypothesis.
z_score: 3.76 Reject null hypothesis.
z_score: 4.64 Reject null hypothesis.
z_score: 3.66 Reject null hypothesis.
z_score: 4.69 Reject null hypothesis.
z_score: 4.13 Reject null hypothesis.


In [18]:
# question 3

In [19]:
p_0 = mean_a
p_1 = mean_b

In [20]:
# set the boundaries: Set desired Type I error = 5% and Type II error = 20%

In [21]:
A = 2.99
B = -1.6

In [22]:
x_1 =math.log(mean_b/mean_a)
x_0 = math.log((1-mean_b)/(1-mean_a))
print(x_1,x_0)

0.16581527647676866 -0.032245902839488846


In [23]:
stopping_times_list = []
for i in range(10,20):
    values = []
    stopping_times = 1
    x_cum = 0
    treatment_sample = variant_B.sample(n=required_n)
    for sample in treatment_sample['purchase_TF']:
        values.append(sample)
        if sample == 1:
            x_cum = x_cum + x_1
        else:
            x_cum = x_cum + x_0
        if x_cum>= A or x_cum <= B:
            stopping_times_list.append(stopping_times)
            break
        stopping_times = stopping_times + 1

def Get_Average(list):
    sum = 0
    for item in list:
        sum += item
    return sum/len(list)
Get_Average(stopping_times_list)

989.6